In [4]:
# FASE 4_PROYECTO ZTNA_202016907_Proyecto de Grado_ANDRES_YORMAN_AIDA

# CASO DE USO #3: Microsegmentación Automática con VMware NSX

# Requisitos: pip install requests pandas matplotlib networkx

import requests
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
from datetime import datetime, timedelta
import os
import csv
import ipaddress
import ssl
import warnings
warnings.filterwarnings("ignore")

# Configuración de credenciales para NSX
# En un entorno de producción, usamos variables de entorno o un sistema de gestión de secretos
NSX_MANAGER = "nsx.ejemplo.com"
NSX_USERNAME = "admin"
NSX_PASSWORD = "your_nsx_password"

# Función para conectar a NSX Manager API
def connect_to_nsx():
    base_url = f"https://{NSX_MANAGER}/api/v1"
    session = requests.Session()

    # Desactivar verificación de certificados (solo para desarrollo)
    # En producción, usar certificados válidos
    session.verify = False

    # Configurar autenticación básica
    session.auth = (NSX_USERNAME, NSX_PASSWORD)

    # Probar la conexión
    try:
        response = session.get(f"{base_url}/transport-zones")
        response.raise_for_status()
        print("Conexión a NSX Manager establecida correctamente")
        return session, base_url
    except requests.exceptions.RequestException as e:
        print(f"Error al conectar a NSX Manager: {e}")
        return None, None

# Función para obtener flujos de tráfico desde NSX
def get_traffic_flows(session, base_url, time_span_hours=24):
    if not session or not base_url:
        return pd.DataFrame()

    # Calcular rango de tiempo
    end_time = int(datetime.now().timestamp() * 1000)  # Milisegundos epoch
    start_time = int((datetime.now() - timedelta(hours=time_span_hours)).timestamp() * 1000)

    # Construir la solicitud para obtener flujos de tráfico
    flows_url = f"{base_url}/flow-records"
    params = {
        'start_time': start_time,
        'end_time': end_time
    }

    try:
        response = session.get(flows_url, params=params)
        response.raise_for_status()

        flows_data = response.json()

        # Convertir a DataFrame
        flows = []
        for flow in flows_data.get('results', []):
            flows.append({
                'source_ip': flow.get('source_ip', ''),
                'destination_ip': flow.get('destination_ip', ''),
                'source_port': flow.get('source_port', 0),
                'destination_port': flow.get('destination_port', 0),
                'protocol': flow.get('protocol', ''),
                'byte_count': flow.get('byte_count', 0),
                'packet_count': flow.get('packet_count', 0),
                'last_update_time': flow.get('last_update_time', 0)
            })

        df = pd.DataFrame(flows)
        print(f"Se obtuvieron {len(df)} registros de flujo de tráfico")
        return df

    except requests.exceptions.RequestException as e:
        print(f"Error al obtener flujos de tráfico: {e}")

        # Para propósitos de demostración, generar datos de ejemplo si la API falla
        return generate_sample_flows()

# Función para generar datos de flujo de ejemplo (para demostraciones)
def generate_sample_flows(num_flows=1000):
    print("Generando datos de flujo de ejemplo para demostración...")

    # IPs para diferentes segmentos
    web_servers = [f"10.1.1.{i}" for i in range(1, 6)]
    app_servers = [f"10.1.2.{i}" for i in range(1, 11)]
    db_servers = [f"10.1.3.{i}" for i in range(1, 4)]
    client_ips = [f"10.2.1.{i}" for i in range(1, 51)]

    # Puertos comunes
    web_ports = [80, 443]
    app_ports = [8080, 8443, 9000]
    db_ports = [1433, 3306, 5432]

    # Protocolos
    protocols = ['TCP', 'UDP']

    # Generar flujos
    flows = []
    for _ in range(num_flows):
        # Patrones de tráfico realistas
        flow_type = np.random.choice([
            'client_to_web',
            'web_to_app',
            'app_to_db',
            'internal_app',
            'random'
        ], p=[0.4, 0.3, 0.2, 0.05, 0.05])

        if flow_type == 'client_to_web':
            source_ip = np.random.choice(client_ips)
            destination_ip = np.random.choice(web_servers)
            destination_port = np.random.choice(web_ports)
            source_port = np.random.randint(10000, 65000)
        elif flow_type == 'web_to_app':
            source_ip = np.random.choice(web_servers)
            destination_ip = np.random.choice(app_servers)
            destination_port = np.random.choice(app_ports)
            source_port = np.random.randint(10000, 65000)
        elif flow_type == 'app_to_db':
            source_ip = np.random.choice(app_servers)
            destination_ip = np.random.choice(db_servers)
            destination_port = np.random.choice(db_ports)
            source_port = np.random.randint(10000, 65000)
        elif flow_type == 'internal_app':
            source_ip = np.random.choice(app_servers)
            destination_ip = np.random.choice(app_servers)
            while destination_ip == source_ip:
                destination_ip = np.random.choice(app_servers)
            destination_port = np.random.choice(app_ports)
            source_port = np.random.randint(10000, 65000)
        else:  # random
            all_ips = web_servers + app_servers + db_servers + client_ips
            source_ip = np.random.choice(all_ips)
            destination_ip = np.random.choice(all_ips)
            while destination_ip == source_ip:
                destination_ip = np.random.choice(all_ips)
            destination_port = np.random.choice(web_ports + app_ports + db_ports)
            source_port = np.random.randint(10000, 65000)

        protocol = np.random.choice(protocols, p=[0.9, 0.1])
        byte_count = np.random.randint(100, 1000000)
        packet_count = np.random.randint(1, 1000)

        flows.append({
            'source_ip': source_ip,
            'destination_ip': destination_ip,
            'source_port': source_port,
            'destination_port': destination_port,
            'protocol': protocol,
            'byte_count': byte_count,
            'packet_count': packet_count,
            'last_update_time': int(datetime.now().timestamp() * 1000) - np.random.randint(0, 86400000)
        })

    return pd.DataFrame(flows)

# Función para asignar IPs a grupos lógicos basados en patrones de comunicación
def assign_ips_to_groups(flows_df):
    # Crear grafo de comunicaciones
    G = nx.DiGraph()

    # Agregar nodos y aristas
    for _, row in flows_df.iterrows():
        src = row['source_ip']
        dst = row['destination_ip']
        dst_port = row['destination_port']
        protocol = row['protocol']
        weight = row['byte_count']

        # Añadir nodos si no existen
        if not G.has_node(src):
            G.add_node(src, type='unknown')
        if not G.has_node(dst):
            G.add_node(dst, type='unknown')

        # Añadir arista o actualizar peso
        if G.has_edge(src, dst):
            G[src][dst]['weight'] += weight
            if dst_port not in G[src][dst]['ports']:
                G[src][dst]['ports'].append(dst_port)
        else:
            G.add_edge(src, dst, weight=weight, ports=[dst_port], protocol=protocol)

    # Identificar tipos de nodos basados en patrones de tráfico
    # Web servers generalmente reciben tráfico en puertos 80, 443
    # DB servers reciben en puertos de bases de datos
    # App servers tienen patrones mixtos
    web_ports = [80, 443]
    db_ports = [1433, 3306, 5432, 27017, 6379, 9042]
    app_ports = [8080, 8443, 9000, 8008, 8888]

    for node in G.nodes():
        # Contar conexiones entrantes por tipo de puerto
        incoming_edges = list(G.in_edges(node, data=True))
        if not incoming_edges:
            continue

        web_connections = 0
        db_connections = 0
        app_connections = 0

        for _, _, data in incoming_edges:
            for port in data['ports']:
                if port in web_ports:
                    web_connections += 1
                elif port in db_ports:
                    db_connections += 1
                elif port in app_ports:
                    app_connections += 1

        # Asignar tipo basado en mayoría de conexiones
        if web_connections > db_connections and web_connections > app_connections:
            G.nodes[node]['type'] = 'web'
        elif db_connections > web_connections and db_connections > app_connections:
            G.nodes[node]['type'] = 'db'
        elif app_connections > web_connections and app_connections > db_connections:
            G.nodes[node]['type'] = 'app'
        else:
            # Si hay empate o no hay suficiente información, intentar inferir por rango de IP
            try:
                ip_obj = ipaddress.ip_address(node)
                ip_int = int(ip_obj)

                # Inferir tipo basado en rangos (usando el ejemplo de datos generados)
                if str(ip_obj).startswith('10.1.1.'):
                    G.nodes[node]['type'] = 'web'
                elif str(ip_obj).startswith('10.1.2.'):
                    G.nodes[node]['type'] = 'app'
                elif str(ip_obj).startswith('10.1.3.'):
                    G.nodes[node]['type'] = 'db'
                elif str(ip_obj).startswith('10.2.1.'):
                    G.nodes[node]['type'] = 'client'
                else:
                    G.nodes[node]['type'] = 'other'
            except:
                G.nodes[node]['type'] = 'unknown'

    # Agrupar nodos por tipo
    groups = {}
    for node, attr in G.nodes(data=True):
        node_type = attr['type']
        if node_type not in groups:
            groups[node_type] = []
        groups[node_type].append(node)

    return G, groups

# Función para visualizar el grafo de comunicaciones
def visualize_communication_graph(G, groups):
    plt.figure(figsize=(16, 12))

    # Definir colores por tipo de grupo
    color_map = {
        'web': 'skyblue',
        'app': 'lightgreen',
        'db': 'salmon',
        'client': 'yellow',
        'other': 'gray',
        'unknown': 'white'
    }

    # Crear posiciones para el layout
    pos = nx.spring_layout(G, k=0.2, iterations=50)

    # Dibujar nodos por grupos
    for group_type, nodes in groups.items():
        nx.draw_networkx_nodes(
            G, pos,
            nodelist=nodes,
            node_color=color_map.get(group_type, 'white'),
            node_size=300,
            alpha=0.8,
            label=f"{group_type.capitalize()} ({len(nodes)})"
        )

    # Dibujar aristas con grosor proporcional al peso
    edge_weights = [G[u][v]['weight']/1000000 for u, v in G.edges()]
    nx.draw_networkx_edges(
        G, pos,
        width=edge_weights,
        alpha=0.5,
        arrows=True,
        arrowsize=10,
        arrowstyle='-|>'
    )

    # Dibujar etiquetas para un subconjunto de nodos (para evitar sobrecarga visual)
    # Seleccionar los nodos con más conexiones
    top_nodes = sorted(G.nodes(), key=lambda x: G.degree(x), reverse=True)[:20]
    nx.draw_networkx_labels(
        G, pos,
        labels={node: node for node in top_nodes},
        font_size=8
    )

    plt.title("Grafo de Comunicaciones de Red para Microsegmentación", size=16)
    plt.legend(scatterpoints=1)
    plt.axis('off')
    plt.tight_layout()

    # Guardar la visualización
    os.makedirs('nsx_analysis', exist_ok=True)
    plt.savefig('nsx_analysis/communication_graph.png', dpi=300)
    plt.close()

# Función para generar reglas de microsegmentación recomendadas
def generate_segmentation_rules(G, groups):
    rules = []

    # Regla 1: Clientes solo pueden acceder a web servers en puertos web
    if 'client' in groups and 'web' in groups:
        rules.append({
            'name': 'Client to Web',
            'source_group': 'client',
            'destination_group': 'web',
            'service': [80, 443],
            'action': 'ALLOW',
            'direction': 'IN',
            'priority': 1000
        })

    # Regla 2: Web servers solo pueden acceder a app servers en puertos de aplicación
    if 'web' in groups and 'app' in groups:
        app_ports = []
        for u, v, data in G.edges(data=True):
            if G.nodes[u]['type'] == 'web' and G.nodes[v]['type'] == 'app':
                app_ports.extend(data['ports'])

        # Eliminar duplicados y ordenar
        app_ports = sorted(list(set(app_ports)))

        rules.append({
            'name': 'Web to App',
            'source_group': 'web',
            'destination_group': 'app',
            'service': app_ports,
            'action': 'ALLOW',
            'direction': 'IN',
            'priority': 1100
        })

    # Regla 3: App servers solo pueden acceder a DB servers en puertos de base de datos
    if 'app' in groups and 'db' in groups:
        db_ports = []
        for u, v, data in G.edges(data=True):
            if G.nodes[u]['type'] == 'app' and G.nodes[v]['type'] == 'db':
                db_ports.extend(data['ports'])

        # Eliminar duplicados y ordenar
        db_ports = sorted(list(set(db_ports)))

        rules.append({
            'name': 'App to DB',
            'source_group': 'app',
            'destination_group': 'db',
            'service': db_ports,
            'action': 'ALLOW',
            'direction': 'IN',
            'priority': 1200
        })

    # Regla 4: Comunicaciones dentro del mismo grupo (puede personalizarse según necesidades)
    for group_type in groups:
        if group_type in ['app', 'web', 'db'] and len(groups[group_type]) > 1:
            # Encontrar puertos usados internamente
            internal_ports = []
            for u, v, data in G.edges(data=True):
                if G.nodes[u]['type'] == group_type and G.nodes[v]['type'] == group_type:
                    internal_ports.extend(data['ports'])

            # Si no hay puertos detectados específicamente, usar puertos comunes por tipo
            if not internal_ports:
                if group_type == 'web':
                    internal_ports = [80, 443, 8080]
                elif group_type == 'app':
                    internal_ports = [8080, 8443, 9000]
                elif group_type == 'db':
                    internal_ports = [1433, 3306, 5432]

            # Eliminar duplicados y ordenar
            internal_ports = sorted(list(set(internal_ports)))

            rules.append({
                'name': f'Internal {group_type.capitalize()} Communication',
                'source_group': group_type,
                'destination_group': group_type,
                'service': internal_ports,
                'action': 'ALLOW',
                'direction': 'IN',
                'priority': 1300 + (100 * list(groups.keys()).index(group_type))
            })

    # Regla 5: Bloquear todo lo demás (política Zero Trust default deny)
    rules.append({
        'name': 'Default Deny',
        'source_group': 'ANY',
        'destination_group': 'ANY',
        'service': ['ANY'],
        'action': 'DENY',
        'direction': 'IN',
        'priority': 2000
    })

    return rules

# Función para generar configuración NSX para reglas
def generate_nsx_config(rules, groups):
    # Formato NSX para grupos
    nsx_groups = []
    for group_type, ips in groups.items():
        if group_type != 'unknown' and ips:  # No crear grupos para desconocidos o vacíos
            nsx_groups.append({
                'display_name': f"{group_type.capitalize()}_Servers",
                'description': f"Auto-generated group for {group_type} servers",
                'expression': [
                    {
                        'resource_type': 'IPAddressExpression',
                        'ip_addresses': ips
                    }
                ]
            })

    # Formato NSX para reglas
    nsx_rules = []
    for rule in rules:
        if rule['source_group'] == 'ANY' and rule['destination_group'] == 'ANY':
            src_group = {"target_type": "IPv4", "resource_type": "IPAddressExpression", "ip_addresses": ["ANY"]}
            dst_group = {"target_type": "IPv4", "resource_type": "IPAddressExpression", "ip_addresses": ["ANY"]}
        else:
            src_group = {"target_display_name": f"{rule['source_group'].capitalize()}_Servers", "is_valid": True}
            dst_group = {"target_display_name": f"{rule['destination_group'].capitalize()}_Servers", "is_valid": True}

        services = []
        if rule['service'] == ['ANY']:
            services = [{"service_type": "ANY"}]
        else:
            for port in rule['service']:
                services.append({
                    "l4_port_set_entry": {
                        "protocol": "TCP",
                        "destination_ports": [str(port)]
                    }
                })

        nsx_rules.append({
            'display_name': rule['name'],
            'action': rule['action'],
            'direction': rule['direction'],
            'sources': [src_group],
            'destinations': [dst_group],
            'services': services,
            'sequence_number': rule['priority']
        })

    # Estructura completa del documento de configuración
    nsx_config = {
        'groups': nsx_groups,
        'communication_map': {
            'display_name': 'Auto-Generated Microsegmentation',
            'description': 'Communication map generated from traffic analysis',
            'rules': nsx_rules
        }
    }

    return nsx_config

# Función para generar la configuración NSX y escribir a archivos
def save_nsx_config(nsx_config):
    os.makedirs('nsx_analysis', exist_ok=True)

    # Guardar grupos
    with open('nsx_analysis/nsx_groups.json', 'w') as f:
        json.dump(nsx_config['groups'], f, indent=2)

    # Guardar reglas
    with open('nsx_analysis/nsx_rules.json', 'w') as f:
        json.dump(nsx_config['communication_map'], f, indent=2)

    # Guardar configuración completa
    with open('nsx_analysis/nsx_full_config.json', 'w') as f:
        json.dump(nsx_config, f, indent=2)

    print(f"Configuración NSX guardada en el directorio 'nsx_analysis'")
    return 'nsx_analysis/nsx_full_config.json'

# Función para crear una tabla HTML con las reglas recomendadas (para visualización)
def create_rules_html_table(rules):
    html = """
    <!DOCTYPE html>
    <html>
    <head>
        <style>
            body { font-family: Arial, sans-serif; }
            table { border-collapse: collapse; width: 100%; }
            th, td { border: 1px solid #ddd; padding: 8px; text-align: left; }
            th { background-color: #f2f2f2; }
            .allow { color: green; }
            .deny { color: red; }
            tr:nth-child(even) { background-color: #f9f9f9; }
        </style>
    </head>
    <body>
        <h2>Reglas de Microsegmentación Recomendadas</h2>
        <table>
            <tr>
                <th>Nombre</th>
                <th>Prioridad</th>
                <th>Origen</th>
                <th>Destino</th>
                <th>Servicio/Puertos</th>
                <th>Acción</th>
                <th>Dirección</th>
            </tr>
    """

    for rule in rules:
        # Formatear puertos para mostrar
        if rule['service'] == ['ANY']:
            service_str = 'ANY'
        else:
            service_str = ', '.join([str(port) for port in rule['service']])

        # Clase CSS según la acción
        action_class = 'allow' if rule['action'] == 'ALLOW' else 'deny'

        html += f"""
        <tr>
            <td>{rule['name']}</td>
            <td>{rule['priority']}</td>
            <td>{rule['source_group']}</td>
            <td>{rule['destination_group']}</td>
            <td>{service_str}</td>
            <td class="{action_class}">{rule['action']}</td>
            <td>{rule['direction']}</td>
        </tr>
        """

    html += """
        </table>
    </body>
    </html>
    """

    # Guardar la tabla HTML
    os.makedirs('nsx_analysis', exist_ok=True)
    with open('nsx_analysis/segmentation_rules.html', 'w') as f:
        f.write(html)

    return 'nsx_analysis/segmentation_rules.html'

# Función principal
def main():
    print("Iniciando análisis para microsegmentación automática...")

    # 1. Conectar a NSX Manager
    session, base_url = connect_to_nsx()

    # 2. Obtener flujos de tráfico
    flows_df = get_traffic_flows(session, base_url)

    # 3. Asignar IPs a grupos lógicos
    G, groups = assign_ips_to_groups(flows_df)

    # 4. Visualizar el grafo de comunicaciones
    visualize_communication_graph(G, groups)

    # 5. Generar reglas de microsegmentación recomendadas
    rules = generate_segmentation_rules(G, groups)

    # 6. Imprimir resumen de grupos y reglas
    print("\nResumen de grupos identificados:")
    for group_type, ips in groups.items():
        print(f"  - {group_type.capitalize()}: {len(ips)} dispositivos")

    print("\nReglas de microsegmentación recomendadas:")
    for rule in rules:
        service_str = ', '.join([str(s) for s in rule['service']]) if rule['service'] != ['ANY'] else 'ANY'
        print(f"  - {rule['name']}: {rule['source_group']} -> {rule['destination_group']} ({service_str}) - {rule['action']}")

    # 7. Generar configuración NSX
    nsx_config = generate_nsx_config(rules, groups)

    # 8. Guardar configuración
    config_file = save_nsx_config(nsx_config)

    # 9. Crear visualización HTML de reglas
    html_file = create_rules_html_table(rules)

    print(f"\nAnálisis completo. Se procesaron {len(flows_df)} flujos de tráfico.")
    print(f"Archivo de configuración NSX guardado en: {config_file}")
    print(f"Visualización de reglas guardada en: {html_file}")

    # 10. Si se ejecuta en Colab, descargar los archivos generados
    try:
        from google.colab import files
        files.download(config_file)
        files.download(html_file)
        files.download('nsx_analysis/communication_graph.png')
    except:
        print(f"Los archivos se han guardado en el directorio 'nsx_analysis'")

# Punto de entrada principal
if __name__ == "__main__":
    main()

Iniciando análisis para microsegmentación automática...
Error al conectar a NSX Manager: HTTPSConnectionPool(host='nsx.ejemplo.com', port=443): Max retries exceeded with url: /api/v1/transport-zones (Caused by SSLError(SSLError(1, '[SSL: TLSV1_UNRECOGNIZED_NAME] tlsv1 unrecognized name (_ssl.c:1016)')))

Resumen de grupos identificados:

Reglas de microsegmentación recomendadas:
  - Default Deny: ANY -> ANY (ANY) - DENY
Configuración NSX guardada en el directorio 'nsx_analysis'

Análisis completo. Se procesaron 0 flujos de tráfico.
Archivo de configuración NSX guardado en: nsx_analysis/nsx_full_config.json
Visualización de reglas guardada en: nsx_analysis/segmentation_rules.html


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>